In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from matplotlib import pyplot as plt
import pickle
import import_ipynb
import sys
sys.path.append('./../../')
import Portfolio_value as pv

In [2]:
df =  pd.read_csv('./../../data/data.csv')

In [3]:
p_val = np.load('./../../result/revision/revision_interest_rate_subgroup/p_val/p_val_dyna.npy', allow_pickle=True)
w_val = np.load('./../../result/revision/revision_interest_rate_subgroup/w_val/w_val_dyna.npy', allow_pickle=True)

p_val = p_val.reshape(1)
p_val = p_val[0]

w_val = w_val.reshape(1)
w_val = w_val[0]


In [4]:
low = df['RF'].quantile(0.55)
mid = df['RF'].quantile(0.6)
high = df['RF'].quantile(1.0)


In [5]:
df['Date'] = pd.to_datetime(df['Date'])

# Group by year and compute mean
annual_avg = df.groupby(df['Date'].dt.year).mean().reset_index()


In [6]:
low_year_ = list(annual_avg[annual_avg['RF'] < low]['Date'])

In [7]:
mid_year_ = list(annual_avg[annual_avg['RF'] >= low][annual_avg['RF'] < mid]['Date'])

In [8]:
high_year_ = list(annual_avg[annual_avg['RF'] >= mid][annual_avg['RF'] < high]['Date'])

In [9]:
bear_year = np.load('./../../result/year/bear_year.npy', allow_pickle=True).reshape(1)[0]


In [10]:
bear_year

{10: [[1988,
   1989,
   1990,
   1991,
   1992,
   1993,
   1994,
   1995,
   1996,
   1997,
   1998,
   1999,
   2000,
   2001,
   2002,
   2003,
   2007,
   2008,
   2009]],
 15: [[1988,
   1989,
   1990,
   1991,
   1992,
   1993,
   1994,
   1995,
   1996,
   1997,
   1998,
   1999,
   2000,
   2001,
   2002,
   2003,
   2007]],
 20: [[1988,
   1989,
   1990,
   1991,
   1992,
   1993,
   1994,
   1995,
   1996,
   1997,
   1998,
   1999,
   2000,
   2001,
   2002]],
 25: [[1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997]],
 30: [[1988, 1989, 1990, 1991, 1992]]}

In [11]:
bear_year_low = {}

for key, value in bear_year.items():
    # Set intersection between the inner list of bear_year and high_year_
    intersection = list(set(value[0]) & set(low_year_))
    if intersection:
        bear_year_low[key] = [sorted(intersection)]

print(bear_year_low)

{10: [[1999, 2000, 2001, 2002, 2003, 2007, 2008, 2009]], 15: [[1999, 2000, 2001, 2002, 2003, 2007]], 20: [[1999, 2000, 2001, 2002]]}


In [12]:
bear_year_mid = {}

for key, value in bear_year.items():
    # Set intersection between the inner list of bear_year and high_year_
    intersection = list(set(value[0]) & set(mid_year_))
    if intersection:
        bear_year_mid[key] = [sorted(intersection)]

print(bear_year_mid)

{10: [[1993, 1994, 1995, 1996, 1997, 1998]], 15: [[1993, 1994, 1995, 1996, 1997, 1998]], 20: [[1993, 1994, 1995, 1996, 1997, 1998]], 25: [[1993, 1994, 1995, 1996, 1997]]}


In [13]:
bear_year_high = {}

for key, value in bear_year.items():
    # Set intersection between the inner list of bear_year and high_year_
    intersection = list(set(value[0]) & set(high_year_))
    if intersection:
        bear_year_high[key] = [sorted(intersection)]

print(bear_year_high)

{10: [[1988, 1989, 1990, 1991, 1992]], 15: [[1988, 1989, 1990, 1991, 1992]], 20: [[1988, 1989, 1990, 1991, 1992]], 25: [[1988, 1989, 1990, 1991, 1992]], 30: [[1988, 1989, 1990, 1991, 1992]]}


In [14]:
bear_year_high = {}

for key, value in bear_year.items():
    # Set intersection between the inner list of bear_year and high_year_
    intersection = list(set(value[0]) & set(high_year_))
    if intersection:
        bear_year_high[key] = [sorted(intersection)]

print(bear_year_high)

In [15]:
upper_years = [20, 25, 30]

In [16]:
low_perform = {}
mid_perform = {}
high_perform = {}
for upper_year in upper_years:
    low_perform[upper_year] = {
    'dep':[],
    'max':[],
    'under':[],
    'inc':[],
    'beq':[],
    'total':[]
}
    mid_perform[upper_year] = {
    'dep':[],
    'max':[],
    'under':[],
    'inc':[],
    'beq':[],
    'total':[]
}
    
    high_perform[upper_year] = {
    'dep':[],
    'max':[],
    'under':[],
    'inc':[],
    'beq':[],
    'total':[]
}

In [17]:
init_wealth = 350000
k = 0.5
withdraw_rate = 0.06

for upper_year in upper_years:
    for years in bear_year[upper_year] :
        for year in years:
            res = pv.get_metric(year, k, upper_year, withdraw_rate, df, init_wealth, p_val_low, w_val_low)
            if res[2] < upper_year*12 : #고갈 시점이 upper_year보다 작으면 (이미 고갈이 되면)
                low_perform[upper_year]['dep'].append(100)
            elif res[2] >= upper_year*12: # 고갈이 안되면
                low_perform[upper_year]['dep'].append(0)
            low_perform[upper_year]['max'].append(res[3])
            low_perform[upper_year]['under'].append(res[4])
            low_perform[upper_year]['inc'].append(res[1])
            low_perform[upper_year]['beq'].append(res[0])
            low_perform[upper_year]['total'].append(res[1] + res[0])

    for years in bear_year[upper_year]:
        for year in years:
            res = pv.get_metric(year, k, upper_year, withdraw_rate, df, init_wealth, p_val_mid, w_val_mid)
            if res[2] < upper_year*12 : #고갈 시점이 upper_year보다 작으면 (이미 고갈이 되면)
                mid_perform[upper_year]['dep'].append(100)
            elif res[2] >= upper_year*12: # 고갈이 안되면
                mid_perform[upper_year]['dep'].append(0)
            mid_perform[upper_year]['max'].append(res[3])
            mid_perform[upper_year]['under'].append(res[4])
            mid_perform[upper_year]['inc'].append(res[1])
            mid_perform[upper_year]['beq'].append(res[0])
            mid_perform[upper_year]['total'].append(res[1] + res[0])

    for years in bear_year[upper_year]:
        for year in years:
            res = pv.get_metric(year, k, upper_year, withdraw_rate, df, init_wealth, p_val_high, w_val_high)
            if res[2] < upper_year*12 : #고갈 시점이 upper_year보다 작으면 (이미 고갈이 되면)
                high_perform[upper_year]['dep'].append(100)
            elif res[2] >= upper_year*12: # 고갈이 안되면
                high_perform[upper_year]['dep'].append(0)
            high_perform[upper_year]['max'].append(res[3])
            high_perform[upper_year]['under'].append(res[4])
            high_perform[upper_year]['inc'].append(res[1])
            high_perform[upper_year]['beq'].append(res[0])
            high_perform[upper_year]['total'].append(res[1] + res[0])


In [18]:
# Define all dictionaries and indices we want to calculate mean for
dicts = [low_perform, mid_perform, high_perform]
indices = upper_years

# Use list comprehension to compute all means at once
# Store results in lists
a = [np.round(np.mean(d[i]['dep']),1) for i in indices for d in dicts]
b = [np.round(np.mean(d[i]['max']),1) for i in indices for d in dicts]
c = [np.round(np.mean(d[i]['under']),1) for i in indices for d in dicts]
d = [np.round(np.mean(d[i]['inc']),1) for i in indices for d in dicts]
e = [np.round(np.mean(d[i]['beq']),1) for i in indices for d in dicts]
f = [np.round(np.mean(d[i]['total']),1) for i in indices for d in dicts]

# If you need to access the results separately, you can do so by indexing the lists:
# For example, a1 corresponds to a_results[0], a2 corresponds to a_results[1], etc.

In [19]:

print('Depletion Prob. (\%) & & {} & {} & {} & & {} & {} & {} & & {} & {} & {}'.format(a[0], a[1], a[2], a[3], a[4], a[5], a[6], a[7], a[8]))
print('Max. decline (\%) & & {} & {} & {} & & {} & {} & {} & & {} & {} & {}'.format(b[0], b[1], b[2], b[3], b[4], b[5], b[6], b[7], b[8]))
print('Underwater duration (month) & & {} & {} & {} & & {} & {} & {} & & {} & {} & {}'.format(c[0], c[1], c[2], c[3], c[4], c[5], c[6], c[7], c[8]))
print('Total withdrawal (\%) & & {} & {} & {} & & {} & {} & {} & & {} & {} & {}'.format(d[0], d[1], d[2], d[3], d[4], d[5], d[6], d[7], d[8]))
print('Bequest (\%) & & {} & {} & {} & & {} & {} & {} & & {} & {} & {}'.format(e[0], e[1], e[2], e[3], e[4], e[5], e[6], e[7], e[8]))
print('Total withdrawal + Bequest (\%) & & {} & {} & {} & & {} & {} & {} & & {} & {} & {}'.format(f[0], f[1], f[2], f[3], f[4], f[5], f[6], f[7], f[8]))






Depletion Prob. (\%) & & 66.7 & 60.0 & 13.3 & & 100.0 & 100.0 & 40.0 & & 100.0 & 100.0 & 60.0
Max. decline (\%) & & -92.8 & -88.4 & -64.6 & & -100.0 & -100.0 & -89.5 & & -100.0 & -100.0 & -97.7
Underwater duration (month) & & 223.9 & 216.9 & 165.3 & & 288.0 & 286.7 & 275.6 & & 346.8 & 344.8 & 330.2
Total withdrawal (\%) & & 75.2 & 77.8 & 85.8 & & 70.9 & 75.5 & 92.2 & & 68.0 & 71.9 & 92.1
Bequest (\%) & & 7.7 & 12.5 & 41.9 & & 0.0 & 0.0 & 11.5 & & 0.0 & 0.0 & 2.5
Total withdrawal + Bequest (\%) & & 82.9 & 90.2 & 127.7 & & 70.9 & 75.5 & 103.7 & & 68.0 & 71.9 & 94.6
